## CHIRPS COGs
### better-wfp-02-04-01

This application takes a CHIPS products, performs COGs and return the optimized product togheter with a .yml file

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'CHIRPS COGs'),
                ('abstract', 'CHIRPS COGs'),
                ('id', 'ewf-wfp-02-04-01')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

In [ ]:
input_identifiers = ['chirps-v2.0.2019.09.20']

**Input references**

In [ ]:
input_references = ['https://catalog.terradue.com/chirps/search?uid=chirps-v2.0.2019.09.20']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import yaml

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())

from ellip_helpers import *
from uuid import uuid4

import gdal

import cioppy
ciop = cioppy.Cioppy()

## Read the products

### check if all the products have the same track number

In [ ]:
metadata = dict()

try:
    metadata = ciop.search(end_point=input_references[0],
                         params=dict(),
                         output_fields='',
                         model='GeoTime')[0]
    
    print metadata

except Exception as e:
    print('ERROR: could not retrieve products metadata. {}'.format(e))

In [ ]:
metadata

### Copy the input in the local folder

In [ ]:
product_name = input_identifiers[0]+'.tif'

product_name

In [ ]:
os.listdir(os.getcwd())

In [ ]:
product_path = os.path.join(data_path, product_name)

product_path

In [ ]:
path = ciop.copy(product_path, os.getcwd())

path

In [ ]:
os.listdir(os.getcwd())

In [ ]:
os.rename(product_name, '{}_{}'.format('temp', product_name))

In [ ]:
os.listdir(os.getcwd())

In [ ]:
print gdal.Info('temp_' + product_name)

#### Apply Cog

In [ ]:
cog('temp_' + product_name, product_name, clear=True)

In [ ]:
os.listdir(os.getcwd())

In [ ]:
print gdal.Info(input_identifiers[0] + '.tif')

### Extract projection and geo_transform

In [ ]:
ds = gdal.Open(product_name)

In [ ]:
metadata['geo_transform'] = ds.GetGeoTransform()
metadata['projection'] = ds.GetProjection()
ds.FlushCache()

In [ ]:
metadata

### Create yaml file

In [ ]:
yaml_file = {
            'creation_dt':metadata['startdate'],
            'extent':{
                'center_dt':metadata['startdate'],
                'coord':metadata['wkt'],
                'from_dt':metadata['startdate'],
                'to_dt':metadata['enddate'],
                },
            'format':{'name':'GeoTiff'},
            'grid_spatial':{
                'projection':{
                    'geo_ref_points':metadata['geo_transform'],
                    'spatial_reference':metadata['projection'],
                    'valid_data':{
                        'coordinates':metadata['wkt'],
                        'type':'Polygon',
                        },
                    },
                },
            'id': str(uuid4()),
            'image':{
                'bands':{'Band 1':{'path':''}}
            },
            'instrument':{'name':''},
            'lineage': {'source_datasets':'CHIRPS'},
            'platform':{'code':''},
            'processing_level':'ARD',
            'product_type':metadata['title']
        }

In [ ]:
yaml_file

In [ ]:
with open('metadata.yaml', 'w') as outfile:
    yaml.dump(yaml_file, outfile, default_flow_style=False)

### Metadata

In [ ]:
# metadata for product

create_metadata(metadata, metadata['identifier'])

# metadate for yaml file

metadata['identifier'] = 'metadata.yaml'

metadata['title'] = 'metadata yaml file'

create_metadata(metadata, metadata['identifier'])

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.